# 📚 How to Build an Intake-ESM Catalog  

**Author:** Nicole Keeney  
**Creation Date:** March 2025  
**Last Modified:** N/A  

## 📖 Overview  
This notebook provides a step-by-step guide to building an **Intake-ESM** catalog for zarrs in an s3 bucket, including how to use `ecgtools` to create a custom parser and structure datasets for use with `intake-esm`.  

## 🔗 Useful Resources  
- 📄 **Building a Custom Parser with `ecgtools`**:  
  [ecgtools documentation](https://ecgtools.readthedocs.io/en/latest/how-to/use-a-custom-parser.html)  
- 📄 **Creating a Catalog with `intake-esm`**:  
  [Intake-ESM documentation](https://intake-esm.readthedocs.io/en/stable/how-to/build-a-catalog-from-timeseries-files.html)  


In [144]:
import pandas as pd
import s3fs 
import traceback
from tqdm import tqdm
from ecgtools import Builder
from ecgtools.builder import INVALID_ASSET, TRACEBACK

## 🛠️ Guide to the `ecgtools` Builder Class 

The primary function of the `Builder` class is to create catalogs from lists of netCDF files or Zarr stores. It "crawls" through your data directories for you, looking for file paths using the information you have specified. You can refer to the official documentation [here](https://ecgtools.readthedocs.io/en/latest/reference/index.html#builder), but I'll also provide a brief overview and additional notes below. 📚

### 📝 Nicole's notes on the Builder class 

Let’s say you have an **S3 bucket** named `salsa_cities` 💃, which contains a CSV file and two directories: `new_york` and `miami`. Each of these directories contains **Zarr stores** with climate data specific to their locations. It's important to remember that Zarr stores are **not files**; they are storage structures represented as **directories** in the S3 file system. Each Zarr store consists of several nested directories and metadata files. In this case, `precip/`, `x/`, `y/`, and `time/` are directories within the Zarr store.

> **salsa_cities/** <br>
  ├── **new_york/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;├── **precip/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;├── **temp/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;├── **x/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── **y/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── **time/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── *.zattrs* <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── *.zgroup* <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── *.zmetadata* <br>
  └── **miami/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;├── **precip/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;├── **temp/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;├── **x/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── **y/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── **time/** <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── *.zattrs* <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── *.zgroup* <br>
  &nbsp;&nbsp;&nbsp;&nbsp;└── *.zmetadata* <br>
  └── *best_cities_to_salsa_dance.csv* <br>

Each Zarr store includes the following structure: 

- **Data variable directories**: `precip/`, `temp/`
- **Spatial and temporal dimension directories**: `x/`, `y/`, `time/`
- **Metadata files**: `.zattrs`, `.zgroup`, `.zmetadata`

#### 💻 Using the Builder to build a catalog 

Now, let's use the `Builder` class to build a catalog for the Zarr stores in our bucket:

```python
builder = Builder(
    paths=["s3://salsa_cities/"], # A list of paths to the data; this should be the root directory of your data files.
    depth=1, # Maximum depth to crawl for assets.
    exclude_patterns=['**/best_cities_to_salsa_dance.csv'], # List of glob patterns to exclude from crawling.
    include_patterns=['**/.zmetadata'] # List of glob patterns to include when crawling 
)
```

After that, we will "build" the catalog using this Builder object and a custom parser function, and then generate a CSV file and JSON file. More information on these steps are in the code. 

### 🔍 Additional notes on the inputs to Builder 
- **depth**: How far within the paths above should the Builder search for your files? Say we just want to look for the files in `paths=["s3://salsa_cities/new_york/"]`. In this case, our depth would be 0, because the root directory contains our Zarr store in it. 
- **exclude_patterns**: We're not interested in the CSV file, so we ignore it when crawling. You can also add directories here using this formatting: `**/another_directory/**`
- **include_patterns**: Since Zarrs are stores and not individual files, we need to be a bit clever here.
  - We just look at the `.zmetadata` file because it's a terminal file within the root directory of the Zarr store; we don't want the crawler to look into the dimensions or data directories.
  - The Builder class isn't really optimized to work with Zarrs, so we need to avoid it crawling into the other directories within the Zarr store. These other directories will contain the metadata files `.zattrs` and `.zgroup`, but not `.zmetadata`; only the root directory has that file. By setting `include_patterns=['/**.zmetadata']`, we are solving two issues: we are ignoring the nested directories within the root directory of the Zarr (`precip`, `x`, `y`, and `time`), and then returning only a single path for each Zarr (the paths for `.zgroup` and `.zattrs` within the root directory are also ignored). 🎯


## Step 1: Build a custom parser function 
This function extracts information from the filepath, such that it can be used by the `Builder` to generate the function. For the input filepaths, the path to the zarr will include the `.zmetadata` extension, even though the `path` key in the output dictionary will **not** include this file extension; I realize this is confusing, but it's a hacky way to get around some inflexibility in the `Builder` class when working with zarrs. See the section above for more info: **Additional notes on the inputs to Builder: include_patterns**.

In [ ]:
def parse_ae_ren_data(filepath):
    """
    Parses the S3 filepath to extract metadata for climate simulation data.
    
    Extracts information like installation, simulation model, experiment, 
    frequency, variable, grid resolution, and the file path (without `.zmetadata`).

    Parameters
    ----------
    filepath : str
        The S3 URL of the file.

    Returns
    -------
    dict
        A dictionary with parsed metadata:
        - installation, activity_id, institution_id, source_id, experiment_id, 
          table_id, variable_id, grid_label, path.
        If parsing fails, returns a dictionary with the error details:
        - INVALID_ASSET and TRACEBACK.

    Example
    -------
    >>> parse_ae_ren_data('s3://wfclimres/ERA/WRF/EC-Earth3/experiment/precipitation/variable/zarr/file.zmetadata')
    {
        "installation": "WRF",
        "activity_id": "WRF",
        "institution_id": "ERA",
        "source_id": "EC-Earth3",
        "experiment_id": "experiment",
        "table_id": "precipitation",
        "variable_id": "variable",
        "grid_label": "zarr",
        "path": "s3://wfclimres/ERA/WRF/EC-Earth3/experiment/precipitation/variable/zarr/file"
    }

    Notes
    -----
    The `try/except` block handles errors in extracting information from the `filepath`. 
    If the filepath structure does not match the expected format or if any error occurs 
    while splitting the string, the `except` block will capture the exception and return 
    a dictionary with the error message and traceback.
    """
    try:
        # Get the data info from the filepath
        institution_id, installation, source_id, experiment_id, table_id, variable_id, grid_label, _ = filepath.split("s3://wfclimres/")[1].split("/")
        # Remove .zmetadata from the filepath, since the actual path to the zarr doesn't include this 
        filepath = filepath.split(".zmetadata")[0]
    except Exception as e:
        # If an error occurs (e.g., wrong filepath structure), return error details
        return {INVALID_ASSET: filepath, TRACEBACK: traceback.format_exc()}
    
    # Simulation string mapping
    simulation_dict = {
        "ec-earth3": "EC-Earth3",
        "mpi-esm1-2-hr": "MPI-ESM1-2-HR",
        "miroc6": "MIROC6",
        "taiesm1": "TaiESM1",
        "era5": "ERA5"
    }

    # Add filepath info to dictionary
    info = {
        "installation": installation,
        "activity_id": "WRF", 
        "institution_id": "ERA",
        "source_id": simulation_dict[source_id],
        "experiment_id": experiment_id,
        "table_id": table_id,
        "variable_id": variable_id,
        "grid_label": grid_label,
        "path": filepath
    }
    
    return info


## Step 2: Build the catalog 
Using the custom parser, we will create our Builder object and build the catalog. How this is coded up will depend on various inputs. I'll show some different methods below.<br><br>
Each method will use some variation of the following code: 
```python 
# Base Builder object 
b = Builder(paths=["s3://path-to-data-directory"]) 

# Build the catalog using a custom parsing function (you need to define this function for your unique data structure)
b.build(parsing_func=custom_parsing_func)

# Exclude invalid assets and removing duplicate entries
b.clean_dataframe()

# View your build catalog as a dataframe :) 
b.df
```


### Method 1: Feed the Builder allllll the filepaths (no crawling required)
The Builder won't do any crawling of your data bucket, because you've oh so kindly fed it all the filepaths it needs. This method requires **you** to do the crawling beforehand to generate a list of these filepaths. 

Below, I've written a bunch of code to crawl through the renewables s3 bucket and get filepaths for all the files I want to include in the catalog

In [ ]:
fs = s3fs.S3FileSystem()

# Use these to filter the s3 bucket 
installations = ["pv_distributed", "pv_utility", "windpower_offshore", "windpower_onshore"]
source_ids = ["ec-earth3", "miroc6", "mpi-esm1-2-hr", "taiesm1", "era5"]

# Total iterations for tqdm
total_iterations = len(installations) * len(source_ids)

filepaths = [] # Store all filepaths here 
with tqdm(total=total_iterations, desc="Scanning S3", unit="query") as pbar:
    for installation in installations:
        for source_id in source_ids:
            # I think each unique zarr store has a single .zmetadata file associated with it 
            # Use .zmetadata to grab path to the main zarr store 
            # Otherwise you get all the random stuff associated with it (variables, coords, etc) since zarr is a directory, not a single file 
            glob_s3 = fs.glob(f"s3://wfclimres/era/{installation}/{source_id}/**/*.zmetadata")
            zarr_paths = ["s3://"+file.split(".zmetadata")[0] for file in glob_s3] # Remove .zmetadata from the path 
            filepaths += zarr_paths 
            pbar.update(1)  # Update progress bar

print(f"Total files found: {len(filepaths)}")

Scanning S3: 100%|██████████| 20/20 [01:05<00:00,  3.26s/query]

Total files found: 112


Next, feed these filepaths to the Builder. Since these filepaths are already **absolute filepaths**, set the argument ``depth=0``: no crawling required. 

In [146]:
b1 = Builder(paths=filepaths, depth=0, include_patterns=["**/.zmetadata"])
b1.build(parsing_func=parse_ae_ren_data)
b1.clean_dataframe()
b1.df

,installation,activity_id,institution_id,source_id,experiment_id,table_id,variable_id,grid_label,path
0,pv_distributed,WRF,ERA,EC-Earth3,historical,1hr,cf,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
1,pv_distributed,WRF,ERA,EC-Earth3,historical,1hr,gen,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
2,pv_distributed,WRF,ERA,EC-Earth3,historical,day,cf,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
3,pv_distributed,WRF,ERA,EC-Earth3,historical,day,gen,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
4,pv_distributed,WRF,ERA,EC-Earth3,ssp370,1hr,cf,d03,s3://wfclimres/era/pv_distributed/ec-earth3/ss...
...,...,...,...,...,...,...,...,...,...
107,windpower_onshore,WRF,ERA,MPI-ESM1-2-HR,ssp370,1hr,gen,d03,s3://wfclimres/era/windpower_onshore/mpi-esm1-...
108,windpower_onshore,WRF,ERA,TaiESM1,historical,1hr,cf,d03,s3://wfclimres/era/windpower_onshore/taiesm1/h...
109,windpower_onshore,WRF,ERA,TaiESM1,historical,1hr,gen,d03,s3://wfclimres/era/windpower_onshore/taiesm1/h...
110,windpower_onshore,WRF,ERA,TaiESM1,ssp370,1hr,cf,d03,s3://wfclimres/era/windpower_onshore/taiesm1/s...


### Method 2: Feed the Builder a directory containing your files 
In this method, this Builder will do some of the hard work for you by crawling through your directories looking for files matching your specifications. Thanks, Builder!<br><br>

#### Notes on the "depth" Builder input
In this case, the path to a file in our directory looks like this: `"s3://wfclimres/era/pv_distributed/ec-earth3/historical/1hr/cf/d03/"`<br>
But, the path we are giving Builder looks like this: `"s3://wfclimres/era/pv_distributed/"`

Thus, the `depth` for this Builder would  be `5`: We need to crawl through 5 different directories (`"ec-earth3/historical/1hr/cf/d03/"`) beyond the root directory to finally reach our zarr store. 

In [142]:
root_dir = 's3://wfclimres/era/'
installations = ["pv_distributed", "pv_utility", "windpower_offshore", "windpower_onshore"]
exclude_patterns = [
    "**/EC-Earth3/**", 
    "**/ERA5/**", 
    "**/MIROC6/**", 
    "**/MPI-ESM1-2-HR/**", 
    "**TaiESM1/**"
    ]
b2 = Builder(
    paths=[f's3://wfclimres/era/{installation}/' for installation in installations], 
    depth=5, 
    exclude_patterns=exclude_patterns, 
    include_patterns=["**/.zmetadata"]
)
b2.build(parsing_func=parse_ae_ren_data)
b2.clean_dataframe()
b2.df

/Users/nicolekeeney/opt/anaconda3/envs/intake-esm-tools/lib/python3.10/site-packages/ecgtools/builder.py:208: UserWarning: Unable to parse 112 assets. A list of these assets can be found in `.invalid_assets` attribute.
  ).clean_dataframe()


,installation,activity_id,institution_id,source_id,experiment_id,table_id,variable_id,grid_label,path
1,pv_distributed,WRF,ERA,EC-Earth3,historical,1hr,cf,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
3,pv_distributed,WRF,ERA,EC-Earth3,historical,1hr,gen,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
5,pv_distributed,WRF,ERA,EC-Earth3,historical,day,cf,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
7,pv_distributed,WRF,ERA,EC-Earth3,historical,day,gen,d03,s3://wfclimres/era/pv_distributed/ec-earth3/hi...
9,pv_distributed,WRF,ERA,EC-Earth3,ssp370,1hr,cf,d03,s3://wfclimres/era/pv_distributed/ec-earth3/ss...
...,...,...,...,...,...,...,...,...,...
215,windpower_onshore,WRF,ERA,MPI-ESM1-2-HR,ssp370,1hr,gen,d03,s3://wfclimres/era/windpower_onshore/mpi-esm1-...
217,windpower_onshore,WRF,ERA,TaiESM1,historical,1hr,cf,d03,s3://wfclimres/era/windpower_onshore/taiesm1/h...
219,windpower_onshore,WRF,ERA,TaiESM1,historical,1hr,gen,d03,s3://wfclimres/era/windpower_onshore/taiesm1/h...
221,windpower_onshore,WRF,ERA,TaiESM1,ssp370,1hr,cf,d03,s3://wfclimres/era/windpower_onshore/taiesm1/s...


### Method 2b: Feed the Builder your entire bucket (slow and untested)
In theory, you can also just feed the builder the entire bucket and have it crawl through everything. However, this method is really slow-- I haven't actually been patient enough to wait for the code to complete running, so I have it commented out below. I'm leaving it here for documentation's sake, in case it is useful for other s3 buckets in the future. 

#### Notes on the "depth" Builder input
In this case, the path to a file in our directory looks like this: `"s3://wfclimres/era/pv_distributed/ec-earth3/historical/1hr/cf/d03/"`<br>
But, the path we are giving Builder looks like this: `"s3://wfclimres/era/"`

Thus, the `depth` for this Builder would  be `6`: We need to crawl through 6 different directories (`"pv_distributed/ec-earth3/historical/1hr/cf/d03/"`) beyond the root directory to finally reach our zarr store. 

In [ ]:
# exclude_patterns = [
#     "s3://wfclimres/era/derived_products/**",
#     "s3://wfclimres/era/resource_data/**",
#     "s3://wfclimres/era/rsrc_drought/**",
#     "s3://wfclimres/era/tmp/**",
#     "s3://wfclimres/era/data-guide_pv-wind.pdf"
#     "**/EC-Earth3/**", 
#     "**/ERA5/**", 
#     "**/MIROC6/**", 
#     "**/MPI-ESM1-2-HR/**", 
#     "**TaiESM1/**"
#     ]
# b3 = Builder(
#     paths=["s3://wfclimres/era/"], 
#     depth=6, 
#     exclude_patterns=exclude_patterns, 
#     include_patterns=["**/.zmetadata"]
# )
# b3.build(parsing_func=parse_ae_ren_data)
# b3.clean_dataframe()
# b3.df

### Confirm that these methods are equal 
If everything went as expected, methods 1 and 2 should return the same result. 

In [ ]:
# Need to reset the index on the second Builder since it's not ordered appropriately for some reason 
b1.df.equals(b2.df.reset_index(drop=True))

True

## Step 3: Export the catalog 
We need to export `csv` and `json` files associated with our built catalog. We also need to set the data aggregations which are fed into xarray when reading in data using intake

In [ ]:
b1.save(
    name='ae-ren',
    directory='../catalogs/ae-ren/',
    # Column name including filepath
    path_column_name='path',
    # Column name including variables
    variable_column_name='variable',
    # Data file format - could be netcdf or zarr (in this case, netcdf)
    data_format="netcdf",
    # Which attributes to groupby when reading in variables using intake-esm
    groupby_attrs=["component", "stream", "case"],
    # Aggregations which are fed into xarray when reading in data using intake
    aggregations=[
        {'type': 'union', 'attribute_name': 'variable'},
        {
            "type": "join_existing",
            "attribute_name": "time_range",
            "options": {"dim": "time", "coords": "minimal", "compat": "override"},
        },
    ],
)